In [1]:
import pandas as pd

In [2]:
# import emdat excel data
emdat_df = pd.read_excel('../../../Resources/EM-DAT/public_emdat_2024-03-30.xlsx')

# split ISO code from DisNo. and retain the numbers only so we can merge w/ disasterlocations.csv
emdat_df['Disaster Year'] = emdat_df['DisNo.'].str.split('-', expand=True)[0]
emdat_df['Disaster Num'] = emdat_df['DisNo.'].str.split('-', expand=True)[1]
emdat_df['disasterno'] = emdat_df['Disaster Year'] + '-' + emdat_df['Disaster Num']

# narrow down the columns
emdat_df = emdat_df[['disasterno', 'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'Country', 'Region', 'Magnitude', 'Latitude', 'Longitude', 'Start Year', 'Start Month',
                     'Start Day', 'Total Deaths', 'No. Injured', 'Total Affected', 'Insured Damage, Adjusted (\'000 US$)', 'Total Damage, Adjusted (\'000 US$)']]
emdat_df.head()

,disasterno,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,Region,Magnitude,Latitude,Longitude,Start Year,Start Month,Start Day,Total Deaths,No. Injured,Total Affected,"Insured Damage, Adjusted ('000 US$)","Total Damage, Adjusted ('000 US$)"
0,1999-9388,Climatological,Drought,Drought,Djibouti,Africa,NaN,NaN,NaN,2001,6.0,NaN,NaN,NaN,100000.0,NaN,NaN
1,1999-9388,Climatological,Drought,Drought,Sudan,Africa,NaN,NaN,NaN,2000,1.0,NaN,NaN,NaN,2000000.0,NaN,NaN
2,1999-9388,Climatological,Drought,Drought,Somalia,Africa,NaN,NaN,NaN,2000,1.0,NaN,21.0,NaN,1200000.0,NaN,NaN
3,2000-0001,Transport,Road,Road,Angola,Africa,NaN,NaN,NaN,2000,1.0,26.0,14.0,11.0,11.0,NaN,NaN
4,2000-0002,Hydrological,Flood,Riverine flood,Angola,Africa,47000.0,NaN,NaN,2000,1.0,8.0,31.0,NaN,70000.0,NaN,16995.0


In [ ]:
# read in disaster locations data
lat_lng_df = pd.read_csv('../Resources/EM-DAT/pend-gdis-1960-2018-disasterlocations.csv')
lat_lng_df.head()

In [ ]:
# merge emdat w/ disaster locations data on disasterno column
merged_df = emdat_df.merge(lat_lng_df, on='disasterno')

# filter out the columns
merged_df = merged_df[['disasterno', 'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'Country', 'Region', 'geolocation', 'Magnitude', 'latitude', 'longitude', 'Start Year', 'Start Month',
                     'Total Deaths', 'No. Injured', 'Total Affected', 'Insured Damage, Adjusted (\'000 US$)', 'Total Damage, Adjusted (\'000 US$)']]

# rename the columns
merged_df = merged_df.rename(columns={'Insured Damage, Adjusted (\'000 US$)': 'Insured Damage (Adjusted)', 'Total Damage, Adjusted (\'000 US$)': 'Total Damage (Adjusted)', 'disasterno': 'Disaster #', 
                                      'Total Deaths': 'Deaths', 'No. Injured': 'Injuries', 'latitude': 'Lat', 'longitude': 'Lng', 'Start Year': 'Year', 'Start Month': 'Month', 'geolocation': 'Geolocation',
                                      'Disaster Subgroup': 'Subgroup', 'Disaster Type': 'Type', 'Disaster Subtype': 'Subtype'})
# merged_df.info()
merged_df.head()

In [ ]:
# get counts of each column for each disaster type
merged_df.groupby('Type').count()

In [ ]:
# save merged_df to csv and json
merged_df.to_csv('../Resources/DisasterData/emdat_cleaned.csv')
merged_df.to_json('../Resources/DisasterData/emdat_cleaned.json', orient='records', indent=4)

In [ ]:
# filter out extreme temp data and drought data
no_drought_or_extremetemps_df = merged_df.loc[(merged_df['Type'] != 'Drought') & (merged_df['Type'] != 'Extreme temperature')]

# save filtered data to csv and json
no_drought_or_extremetemps_df.to_csv('../Resources/DisasterData/emdat_no_drought_or_xtremetemp.csv')
no_drought_or_extremetemps_df.to_json('../Resources/DisasterData/emdat_no_drought_or_xtremetemps.json', orient='records', indent=4)